## Get lines that may have typographical or grammatical errors

Read rejected.txt and discard lines with other errors

อ่าน rejected.txt และกรองบรรทัดที่มีตัวละติน ไม้ยมก ฯลฯ ทิ้ง
(โดยไอเดียคือควรจะเหลือเฉพาะบรรทัดที่น่าจะมีการสะกดผิด พิมพ์ผิด ผิดไวยากรณ์ หรือมีรูปประโยคในแบบที่เจ้าของภาษาไม่น่าจะใช้)

- main repo: https://github.com/wannaphong/th-cv-sentences
- filter functions from: https://github.com/common-voice/sentence-collector/tree/main/server/lib/validation

In [1]:
# Filter functions from
# https://github.com/common-voice/sentence-collector/tree/main/server/lib/validation
    
# Notes
# - Thai Unicode range: \u0E00-\u0E7F
# - Thai sentence is written without space between words.
# See discussion here:
# https://github.com/Common-Voice/sentence-collector/issues/318

# We count chars to validate instead of words.
# Target max time length for recorded speech: 7-10 seconds
MIN_LENGTH = 2;
MAX_LENGTH = 110;

# Numbers that are not allowed in a sentence depending on the language. For
# English this is 0-9 once or multiple times after each other.
# Thai digits: \u0E50-\u0E59 (๐-๙)
NUMBERS_REGEX = r"[0-9๐-๙]+"

# Some languages want to check the structure, this is what this REGEX is for.
#
# Sentence with running characters of 80 or more without a space is considered difficult to read.
#
# These classes of Thai characters are not allowed to be immediately repeated:
# - Lead vowels: \u0E40\u0E41\u0E42\u0E43\u0E44
# - Follow vowels: \u0E30\u0E32\u0E33\u0E45
# - Above vowels: \u0E31\u0E34\u0E35\u0E36\u0E37\u0E4D\u0E47
# - Below vowels: \u0E38\u0E39
# - Tone marks: \u0E48\u0E49\u0E4A\u0E4B
# - Phinthu: \u0E3A
# - Thanthakhat: \u0E4C
# - Nikhahit: \u0E4D
# - Yamakkan: \u0E4E
#
# These classes of Thai characters have a specific legitimate order.
# - Tone marks/Pinthu/Thanthakat/Nikhahit/Yamakkan can't immediately come after lead and follow vowels
# - Tone marks/Pinthu/Thanthakat/Nikhahit/Yamakkan can't immediately come before above and below vowels
STRUCTURE_REGEX = r"[\u0E01-\u0E4Ea-zA-Z.,\-\"'?!:;]{80,}|[\u0E40\u0E41\u0E42\u0E43\u0E44]{2,}|\u0E30{2,}|[\u0E32\u0E33\u0E45]{2,}|[\u0E31\u0E34\u0E35\u0E36\u0E37\u0E4D\u0E47]{2,}|[\u0E38\u0E39]{2,}|[\u0E48\u0E49\u0E4A\u0E4B]{2,}|\u0E3A{2,}|\u0E4C{2,}|\u0E4D{2,}|\u0E4E{2,}|[\u0E40\u0E41\u0E42\u0E43\u0E44\u0E30\u0E32\u0E33\u0E45][\u0E48\u0E49\u0E4A\u0E4B\u0E3A\u0E4C\u0E4D\u0E4E]|[\u0E48\u0E49\u0E4A\u0E4B\u0E3A\u0E4C\u0E4D\u0E4E][\u0E31\u0E34\u0E35\u0E36\u0E37\u0E4D\u0E47\u0E38\u0E39]"

# These Thai chars cannot start the word:
# - All vowels except lead vowels
# - Tone marks
# - Phinthu, Thanthakhat, Nikhahit, Yamakkan
BEGIN_REGEX = r"(^|\s+)[\u0E30\u0E32\u0E33\u0E45\u0E31\u0E34\u0E35\u0E36\u0E37\u0E4D\u0E47\u0E38\u0E39\u0E48\u0E49\u0E4A\u0E4B\u0E3A\u0E4C\u0E4D\u0E4E]"

# These Thai chars cannot end the word:
# - Lead vowels
END_REGEX = r"[\u0E40\u0E41\u0E42\u0E43\u0E44](\s+|$)"

# The following symbols are disallowed, please update here as well and not just the regex
# to make it easier to read:
# < > + * \ # @ ^ [ ] ( ) /
# Paiyannoi: \u0E2F ฯ (ellipsis, abbreviation)
# Maiyamok: \u0E46 ๆ (repetition)
# Fongman: \u0E4F ๏ (used as bullet)
# Angkhankhu: \u0E5A ๚ (used to mark end of section/verse)
# Khomut: \u0E5B ๛ (used to mark end of chapter/document)
#
# Latin characters are disallowed as well,
# as they can introduce difficulty for pronunciation.
SYMBOL_REGEX = r"[<>+*\\#@^[\]()/\u0E2F\u0E46\u0E4F\u0E5A\u0E5B]|[A-Za-z]+"

# Any words consisting of uppercase letters or uppercase letters with a period
# inbetween are considered abbreviations or acronyms.
# This currently also matches fooBAR but we most probably don't want that either
# as users wouldn't know how to pronounce the uppercase letters.
ABBREVIATION_REGEX = r"[A-Z]{2,}|[A-Z]+\.*[A-Z]+|[ก-ฮ]+\.([ก-ฮ]+\.)+"

In [2]:
import re

def filterNumbers(sentence):
    return not re.search(NUMBERS_REGEX, sentence)

def filterAbbreviations(sentence):
    return not re.search(ABBREVIATION_REGEX, sentence)

def filterSymbols(sentence):
    return not re.search(SYMBOL_REGEX, sentence)

def filterStructure(sentence):
    return (not re.search(STRUCTURE_REGEX, sentence)) or re.search(BEGIN_REGEX, sentence) or re.search(END_REGEX, sentence)

def filterLength(sentence):
    return len(sentence) >= MIN_LENGTH and len(sentence) <= MAX_LENGTH

def isValid(sentence):
    if filterLength(sentence):
        if filterNumbers(sentence) and filterAbbreviations(sentence) and filterSymbols(sentence):
            return True
    return False

In [ ]:
FILENAME = "rejected.txt"

with open(FILENAME, "r") as reader:
    for line in reader.readlines():
        if isValid(line):
            print(line, end="")